In [72]:
# Import dependecies
import pandas as pd
import numpy as np
import difflib
import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [73]:
movies_df = pd.read_csv('../data/movies.csv')
movies_df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [74]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   index                 4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4775 non-null   object 
 3   homepage              1712 non-null   object 
 4   id                    4803 non-null   int64  
 5   keywords              4391 non-null   object 
 6   original_language     4803 non-null   object 
 7   original_title        4803 non-null   object 
 8   overview              4800 non-null   object 
 9   popularity            4803 non-null   float64
 10  production_companies  4803 non-null   object 
 11  production_countries  4803 non-null   object 
 12  release_date          4802 non-null   object 
 13  revenue               4803 non-null   int64  
 14  runtime               4801 non-null   float64
 15  spoken_languages     

In [75]:
movies_df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [76]:
selected_features = ['genres', 'keywords', 'tagline', 'cast', 'director', 'production_companies']

In [77]:
# Replacing null values with null string
for feature in selected_features:
    movies_df[feature] = movies_df[feature].fillna('')


In [78]:
# combining all the selected_features
combined_df = movies_df['genres'] + movies_df['keywords'] + movies_df['tagline'] + movies_df['cast'] + movies_df['director'] + movies_df['production_companies']
combined_df

0       Action Adventure Fantasy Science Fictioncultur...
1       Adventure Fantasy Actionocean drug abuse exoti...
2       Action Adventure Crimespy based on novel secre...
3       Action Crime Drama Thrillerdc comics crime fig...
4       Action Adventure Science Fictionbased on novel...
                              ...                        
4798    Action Crime Thrillerunited states\u2013mexico...
4799    Comedy RomanceA newlywed couple's honeymoon is...
4800    Comedy Drama Romance TV Moviedate love at firs...
4801    A New Yorker in ShanghaiDaniel Henney Eliza Co...
4802    Documentaryobsession camcorder crush dream gir...
Length: 4803, dtype: object

In [79]:
vectorizer = TfidfVectorizer()

In [80]:
feature_vectors = vectorizer.fit_transform(combined_df)
print(feature_vectors)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 165404 stored elements and shape (4803, 35849)>
  Coords	Values
  (0, 5300)	0.06658757622152417
  (0, 5506)	0.07719799906673884
  (0, 14900)	0.10006023738585378
  (0, 29328)	0.08429846010963403
  (0, 15290)	0.2323090586057348
  (0, 10029)	0.18492298683739533
  (0, 16317)	0.13828194096156365
  (0, 30556)	0.28384313631066155
  (0, 34479)	0.11065020650212594
  (0, 10280)	0.2032544217247949
  (0, 30393)	0.2323090586057348
  (0, 32012)	0.05976314733592446
  (0, 35360)	0.10525869838053162
  (0, 25214)	0.07178881654153738
  (0, 25680)	0.18084621464621511
  (0, 28984)	0.1325320943157888
  (0, 35384)	0.1991776495336147
  (0, 35803)	0.16771711203655737
  (0, 28947)	0.18280612374528368
  (0, 30031)	0.17141335003368377
  (0, 34703)	0.16659155194999575
  (0, 30947)	0.13645730924289162
  (0, 21140)	0.18974478492108335
  (0, 23447)	0.13314716087943707
  (0, 28145)	0.2323090586057348
  :	:
  (4801, 18861)	0.3167733394535455
  (4802, 24545)	

In [81]:
# Cosine Similarity: getting the similarity confidence value
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.0683712  0.03194165 ... 0.02178793 0.         0.02108254]
 [0.0683712  1.         0.03873024 ... 0.02164503 0.         0.01241619]
 [0.03194165 0.03873024 1.         ... 0.01367098 0.         0.01068244]
 ...
 [0.02178793 0.02164503 0.01367098 ... 1.         0.         0.01118443]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.02108254 0.01241619 0.01068244 ... 0.01118443 0.         1.        ]]


In [ ]:
# Getting input from user
movie_name = 'Avatar'

In [83]:
movie_name

'toy'

In [84]:
# Create a list with all the list of the movie names given in the dataset
list_of_all_titles = movies_df['title'].tolist()

In [85]:
find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
find_close_matches

['Joy', 'Anatomy']

In [86]:
close_match = find_close_matches[0]

In [87]:
idx_of_movie = movies_df[movies_df.title == close_match]['index'].values[0]
idx_of_movie

np.int64(738)

In [88]:
similarity_score = list(enumerate(similarity[idx_of_movie]))
similarity_score

[(0, np.float64(0.05301655967545137)),
 (1, np.float64(0.02392728203240403)),
 (2, np.float64(0.050098019269020924)),
 (3, np.float64(0.04830635046840521)),
 (4, np.float64(0.02259834975960856)),
 (5, np.float64(0.02737198771422932)),
 (6, np.float64(0.020076192214526813)),
 (7, np.float64(0.05342780536311941)),
 (8, np.float64(0.015972851909749855)),
 (9, np.float64(0.07510188166870135)),
 (10, np.float64(0.0338996583142176)),
 (11, np.float64(0.007356495651362514)),
 (12, np.float64(0.026561859155708425)),
 (13, np.float64(0.03880821072328814)),
 (14, np.float64(0.06522305781390678)),
 (15, np.float64(0.04640698850613459)),
 (16, np.float64(0.0539562956608417)),
 (17, np.float64(0.02599561566540508)),
 (18, np.float64(0.04377251208795717)),
 (19, np.float64(0.03446667215671252)),
 (20, np.float64(0.03370175572170401)),
 (21, np.float64(0.056848479494199175)),
 (22, np.float64(0.031734870546632564)),
 (23, np.float64(0.024447796096585987)),
 (24, np.float64(0.038564394613759885)),
 (2

In [89]:
sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)
sorted_similar_movies

[(738, np.float64(1.0)),
 (1149, np.float64(0.215433540049925)),
 (2071, np.float64(0.18153305293632474)),
 (3355, np.float64(0.17572341305671108)),
 (2838, np.float64(0.17464222271180116)),
 (64, np.float64(0.1716237261642344)),
 (368, np.float64(0.1620143699135024)),
 (1187, np.float64(0.16047849893352578)),
 (359, np.float64(0.15867951718359097)),
 (884, np.float64(0.15521679141132944)),
 (2566, np.float64(0.15483096042998248)),
 (1952, np.float64(0.15068220612087915)),
 (2587, np.float64(0.14977512647723212)),
 (292, np.float64(0.14944270979421478)),
 (1305, np.float64(0.1473845499180696)),
 (46, np.float64(0.14325342365422894)),
 (2273, np.float64(0.14222313632804667)),
 (1845, np.float64(0.1392049963652402)),
 (1682, np.float64(0.13498276722108263)),
 (1111, np.float64(0.13496365209617042)),
 (1147, np.float64(0.1303818914704637)),
 (1502, np.float64(0.12797100599768704)),
 (563, np.float64(0.12344015912967449)),
 (2824, np.float64(0.12309415117498851)),
 (82, np.float64(0.122863

In [90]:
print('Movies suggested for you: \n')

i = 1
for i in range(30):
    index = sorted_similar_movies[i][0]
    title_from_index = movies_df[movies_df.index == index]['title'].values[0]
    print(f'{i} : {title_from_index}')

Movies suggested for you: 

0 : Joy
1 : American Hustle
2 : Silver Linings Playbook
3 : Devil's Due
4 : The Fault in Our Stars
5 : X-Men: Apocalypse
6 : Percy Jackson: Sea of Monsters
7 : Bridge of Spies
8 : Alvin and the Chipmunks: The Road Chip
9 : Zero Dark Thirty
10 : All About Steve
11 : The Counselor
12 : Killing Them Softly
13 : Eragon
14 : Water for Elephants
15 : X-Men: Days of Future Past
16 : Hands of Stone
17 : Hide and Seek
18 : Sausage Party
19 : Victor Frankenstein
20 : The Big Year
21 : Men of Honor
22 : Doctor Dolittle
23 : Chronicle
24 : Dawn of the Planet of the Apes
25 : American Sniper
26 : Deadpool
27 : Taken 3
28 : Body of Lies
29 : Bride Wars


In [91]:
# Single Function
def recommend_movie(movie_name):
    list_of_all_titles = movies_df['title'].tolist()
    find_close_matches = difflib.get_close_matches(movie_name, list_of_all_titles)
    close_match = find_close_matches[0]
    idx_of_movie = movies_df[movies_df.title == close_match]['index'].values[0]

    similarity_score = list(enumerate(similarity[idx_of_movie]))
    sorted_similar_movies = sorted(similarity_score, key=lambda x: x[1], reverse=True)

    print('Movies suggested for you: \n')

    i = 1
    for i in range(30):
        index = sorted_similar_movies[i][0]
        title_from_index = movies_df[movies_df.index == index]['title'].values[0]
        print(f'{i} : {title_from_index}')


recommend_movie(movie_name = input('Enter your favorite movie name: '))
    

Movies suggested for you: 

0 : Joy
1 : American Hustle
2 : Silver Linings Playbook
3 : Devil's Due
4 : The Fault in Our Stars
5 : X-Men: Apocalypse
6 : Percy Jackson: Sea of Monsters
7 : Bridge of Spies
8 : Alvin and the Chipmunks: The Road Chip
9 : Zero Dark Thirty
10 : All About Steve
11 : The Counselor
12 : Killing Them Softly
13 : Eragon
14 : Water for Elephants
15 : X-Men: Days of Future Past
16 : Hands of Stone
17 : Hide and Seek
18 : Sausage Party
19 : Victor Frankenstein
20 : The Big Year
21 : Men of Honor
22 : Doctor Dolittle
23 : Chronicle
24 : Dawn of the Planet of the Apes
25 : American Sniper
26 : Deadpool
27 : Taken 3
28 : Body of Lies
29 : Bride Wars


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   genres        4775 non-null   object 
 1   title         4803 non-null   object 
 2   cast          4760 non-null   object 
 3   release_date  4802 non-null   object 
 4   director      4773 non-null   object 
 5   vote_average  4803 non-null   float64
 6   popularity    4803 non-null   float64
dtypes: float64(2), object(5)
memory usage: 262.8+ KB


,genres,title,cast,release_date,director,vote_average,popularity
0,Action Adventure Fantasy Science Fiction,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,2009,James Cameron,7.2,150.437577
1,Adventure Fantasy Action,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,2007,Gore Verbinski,6.9,139.082615
2,Action Adventure Crime,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,2015,Sam Mendes,6.3,107.376788
3,Action Crime Drama Thriller,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,2012,Christopher Nolan,7.6,112.312950
4,Action Adventure Science Fiction,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,2012,Andrew Stanton,6.1,43.926995
...,...,...,...,...,...,...,...
4798,Action Crime Thriller,El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,1992,Robert Rodriguez,6.6,14.269792
4799,Comedy Romance,Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,2011,Edward Burns,5.9,0.642552
4800,Comedy Drama Romance TV Movie,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,2013,Scott Smith,7.0,1.444476
4801,NaN,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,2012,Daniel Hsia,5.7,0.857008


genres          28
title            0
cast            43
release_date     0
director        30
vote_average     0
popularity       0
dtype: int64

genres          0
title           0
cast            0
release_date    0
director        0
vote_average    0
popularity      0
dtype: int64

,genres,title,cast,release_date,director,vote_average,popularity,new_title
0,Action Adventure Fantasy Science Fiction,Avatar,Sam Worthington Zoe Saldana Sigourney Weaver S...,2009,James Cameron,7.2,150.437577,Avatar 2009
1,Adventure Fantasy Action,Pirates of the Caribbean: At World's End,Johnny Depp Orlando Bloom Keira Knightley Stel...,2007,Gore Verbinski,6.9,139.082615,Pirates of the Caribbean: At World's End 2007
2,Action Adventure Crime,Spectre,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,2015,Sam Mendes,6.3,107.376788,Spectre 2015
3,Action Crime Drama Thriller,The Dark Knight Rises,Christian Bale Michael Caine Gary Oldman Anne ...,2012,Christopher Nolan,7.6,112.312950,The Dark Knight Rises 2012
4,Action Adventure Science Fiction,John Carter,Taylor Kitsch Lynn Collins Samantha Morton Wil...,2012,Andrew Stanton,6.1,43.926995,John Carter 2012
...,...,...,...,...,...,...,...,...
4798,Action Crime Thriller,El Mariachi,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,1992,Robert Rodriguez,6.6,14.269792,El Mariachi 1992
4799,Comedy Romance,Newlyweds,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,2011,Edward Burns,5.9,0.642552,Newlyweds 2011
4800,Comedy Drama Romance TV Movie,"Signed, Sealed, Delivered",Eric Mabius Kristin Booth Crystal Lowe Geoff G...,2013,Scott Smith,7.0,1.444476,"Signed, Sealed, Delivered 2013"
4801,,Shanghai Calling,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,2012,Daniel Hsia,5.7,0.857008,Shanghai Calling 2012


,genres,cast,director,vote_average,popularity,new_title
0,Action Adventure Fantasy Science Fiction,Sam Worthington Zoe Saldana Sigourney Weaver S...,James Cameron,7.2,150.437577,Avatar 2009
1,Adventure Fantasy Action,Johnny Depp Orlando Bloom Keira Knightley Stel...,Gore Verbinski,6.9,139.082615,Pirates of the Caribbean: At World's End 2007
2,Action Adventure Crime,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,Sam Mendes,6.3,107.376788,Spectre 2015
3,Action Crime Drama Thriller,Christian Bale Michael Caine Gary Oldman Anne ...,Christopher Nolan,7.6,112.312950,The Dark Knight Rises 2012
4,Action Adventure Science Fiction,Taylor Kitsch Lynn Collins Samantha Morton Wil...,Andrew Stanton,6.1,43.926995,John Carter 2012
...,...,...,...,...,...,...
4798,Action Crime Thriller,Carlos Gallardo Jaime de Hoyos Peter Marquardt...,Robert Rodriguez,6.6,14.269792,El Mariachi 1992
4799,Comedy Romance,Edward Burns Kerry Bish\u00e9 Marsha Dietlein ...,Edward Burns,5.9,0.642552,Newlyweds 2011
4800,Comedy Drama Romance TV Movie,Eric Mabius Kristin Booth Crystal Lowe Geoff G...,Scott Smith,7.0,1.444476,"Signed, Sealed, Delivered 2013"
4801,,Daniel Henney Eliza Coupe Bill Paxton Alan Ruc...,Daniel Hsia,5.7,0.857008,Shanghai Calling 2012


,genres,cast,director,vote_average,popularity,new_title
68,Action Science Fiction Adventure,Robert Downey Jr. Terrence Howard Jeff Bridges...,Jon Favreau,7.4,120.725053,Iron Man 2008
31,Action Adventure Science Fiction,Robert Downey Jr. Gwyneth Paltrow Don Cheadle ...,Shane Black,6.8,77.682080,Iron Man 3 2013
79,Adventure Action Science Fiction,Robert Downey Jr. Gwyneth Paltrow Don Cheadle ...,Jon Favreau,6.6,77.300194,Iron Man 2 2010
913,Comedy,Jim Carrey Zooey Deschanel Rhys Darby Sasha Al...,Peyton Reed,6.4,43.849351,Yes Man 2008
1814,Drama History,Josh Brolin Elizabeth Banks Ellen Burstyn Ioan...,Oliver Stone,6.1,10.445391,W. 2008
2680,History Drama,Meryl Streep Anthony Stewart Head Harry Lloyd ...,Phyllida Lloyd,6.2,23.378400,The Iron Lady 2011
977,Adventure Animation Family Fantasy Science Fic...,Eli Marienthal Jennifer Aniston Vin Diesel Chr...,Brad Bird,7.6,61.245957,The Iron Giant 1999
1380,Action Adventure Drama,Leonardo DiCaprio Jeremy Irons G\u00e9rard Dep...,Randall Wallace,6.3,28.980270,The Man in the Iron Mask 1998
2602,Action,Russell Crowe Lucy Liu Jamie Chung RZA Zhu Zhu,RZA,5.2,17.672021,The Man with the Iron Fists 2012
2221,Action Comedy Crime,Samuel L. Jackson Eugene Levy Luke Goss Miguel...,Les Mayfield,5.4,6.925645,The Man 2005
